In [1]:
import pandas as pd
import numpy as np
import pickle
from bs4 import BeautifulSoup

In [2]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [3]:
course_df.head(1)

,course_id,course_name,course_price,teacher_id,teacher_intro,groups,sub_groups,groups+subgroups,topics,course_published_at_local,description,will_learn,required_tools,recommended_background,target_group
0,61888e868f154b000781b45a,少女人妻華麗變身：七大妝容七彩的夢幻樂園,1800,61888e7bb7fe1c0006850eff,在美妝 KOL 圈裡屬個人風格強烈的 Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影...,生活品味,"更多生活品味,護膚保養與化妝","生活品味_更多生活品味,生活品味_護膚保養與化妝","更多生活品味,護膚保養與化妝",NaN,"<img src=""https://images.api.hahow.in/images/6...",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,所需工具為：視課程實際會用到的彩妝用品,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️ 雖然課程當中會帶到相關彩妝技巧，不...,熱愛彩妝的人


# 課程 Vectorize

In [4]:
id2course_mapping = course_df["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)


728

In [5]:
#course_df = course_df.drop(['teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
for i in range(728):
    s = ''
    s = course_df['description'][i]
    s1 = BeautifulSoup(str(s) ,'lxml').text
    course_df['description'][i] = s1

/tmp/ipykernel_2033/4009512890.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_df['description'][i] = s1


In [6]:
course_df = course_df.replace('\n', '',regex=True).replace('&.;', '',regex=True).replace("--&?", "",regex=True)

In [7]:
def clean_data(x):
    return x.replace(" ", "")

fillna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'groups', 'sub_groups', 'topics', 'will_learn', 'recommended_background', 'target_group', 'required_tools' ,'description','groups+subgroups']
fillna=fillna[features]
for feature in features:
    fillna[feature] = fillna[feature].apply(clean_data)
    
fillna.head(1)

,course_name,teacher_intro,groups,sub_groups,topics,will_learn,recommended_background,target_group,required_tools,description,groups+subgroups
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,生活品味,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。,只要你有一顆愛化妝、想變漂亮的心皆可以參加。⚠️雖然課程當中會帶到相關彩妝技巧，不過內容偏向...,熱愛彩妝的人,所需工具為：視課程實際會用到的彩妝用品,少女人妻第一堂線上課程，跟著我一起華麗變身！在社群上許多人經常問我：「為什麼人妻畫的妝那麼好...,"生活品味_更多生活品味,生活品味_護膚保養與化妝"


In [30]:
#import jieba.analyse
# 想說可用jieba之類的，但不一定會比較好
def create_combination(x):
    course_name = x['course_name']
    teacher_intro = x['teacher_intro']
    will_learn = x['will_learn']
    recommended_background = x['recommended_background']
    target_group = x['target_group']
    required_tools = x['required_tools']
    group = x['groups']
    text = course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] +' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
    #text = course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['groups+subgroups']

    return text

fillna['combination'] = fillna.apply(create_combination, axis=1)
fillna['combination'].head(5)

0    少女人妻華麗變身：七大妝容七彩的夢幻樂園 在美妝KOL圈裡屬個人風格強烈的Alice，在清新...
1    幾何圖形分割X色塊組合 從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現...
2    數位拼貼的手感 自由工作者，致力於品牌視覺規劃，包含識別系統設計、平面設計、包裝設計、網頁、...
3    Line的貼圖自己動手做！ 我是Danny，現是一名自由接案的插畫師，主要是做插畫設計、人物...
4    為申請學校或工作寫好英文自傳 在北美長大，回到南台灣尋根後，現落腳於南加州的庄腳嬰仔。在鑽研...
Name: combination, dtype: object

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import datetime
from scipy.sparse import csr_matrix
import jieba
import re

In [32]:
def tokenize(sentence):
    without_duplicates = re.sub(r'(.)\1+', r'\1\1', sentence)
    #ws_token = ws(without_duplicates)
    return jieba.lcut(without_duplicates)

In [479]:
se = '少女人妻華麗變身：七大妝容七彩的夢幻樂園 在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法引起許多粉絲的共鳴，因此廣受廠商與粉絲喜愛。 生活品味 生活品味 生活品味 更多生活品味,護膚保養與化妝 少女人妻第一堂線上課程，跟著我一起華麗變身！在社群上許多人經常問我：「為什麼人妻畫的妝那麼好看？」、「為什麼任何顏色在人妻臉上都那麼合理？」，而這些用色的妝容秘密，都即將在這堂課程中揭曉。我的美妝創作就是尋找生活中的不同靈感，將臉當作畫布，再用彩妝將它呈現。透過本堂課程的開設，希望讓同學能夠活用各種顏色，透過分享我的化妝技巧，帶你畫出七大色系的妝容，完成你自己最想要的妝容定調！讓你掌握七大色系的美妝應用，消滅對顏色的使用疑慮七大色系的色彩揮灑，讓你在忙碌的日常生活中也不鬆懈自己的美貌！化妝並不是為了誰，而是要美給自己看，不論妳幾歲，愛美的心都不能被消滅，這堂課程希望能夠消滅你對顏色的疑慮，勇敢大膽地去揮灑色彩、精采生活，讓你的美貌有更多可能性！在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法引起許多粉絲的共鳴，因此廣受廠商與粉絲喜愛。Facebook粉專：What\'sthePinklio少女人妻的粉紅玩具Instagram：@alice_ko_depp七大妝容色系切入，邁向「零廢色」的彩妝世界在化妝這條路上，你是不是會碰到這樣子的問題呢：主流色系看了好膩，每年熱門妝容也總是千篇一律每盤眼影當中，總有1～2個顏色看了就怯步，不敢嘗試雖然已有化妝底子，卻沒有個人特色，難以發展風格繽紛多彩的彩妝設計雖然真的很迷人，常常讓人不小心手滑，不過在把新奇戰利品帶回家時，如果無法掌握使用之道，還是很有可能會不停踩雷。這堂課程就是想透過七大妝容的示範，讓你可以從中領會箇中之道！更詳盡的美妝手法教學，成就你想要的樣子「解鎖那些你不敢用的顏色，化妝就是讓自己變漂亮！」比起少女人妻的YouTube頻道影片或者短影片教學，這堂課程將會從七大妝容、七大色系切入教學，期待你上完這堂課，完全進入「完全零廢色」的彩妝世界，我們需要思考的，是什麼樣的方式能夠讓妳完成想要的妝容，最適合自己、讓自己更美麗的妝才是最重要的！七個色系課程單元，從技法到顏色一次掌握！紅色妝容-姐畫的不是妝，是自信：日常必備的自然紅潤好氣色➤重點技巧如何評估自己的肌膚色系'
without_duplicates = re.sub(r'(.)\1+', r'\1\1', se)
#ws_token = ws()
len(jieba.lcut(without_duplicates))

603

In [33]:
tfidf = TfidfVectorizer(tokenizer=tokenize,stop_words='english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(fillna['combination'])

# TF-IDF Dicitonary
#dictionary = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))
#print(tfidf.vocabulary_)
#print(tfidf_matrix)
tfidf_matrix_nd = tfidf_matrix.toarray()
np.save('tfidf_embedding/item_embeddings_jieba_english_all.npy', tfidf_matrix_nd)

In [34]:
print(len(tfidf.get_feature_names_out()))
list(tfidf.vocabulary_.items())[:2]

14650


[('少女', 5849), ('人', 2368)]

In [35]:
from scipy.sparse import csr_matrix
item_embed_FM = np.load('tfidf_embedding/item_embeddings_jieba_english_all.npy')
item_embed_FM = item_embed_FM[:728]
print(tfidf_matrix.shape)
print(item_embed_FM.shape)

(728, 14650)
(728, 14650)


In [36]:
concatenate_item_embed_nd = np.concatenate([tfidf_matrix_nd, item_embed_FM/100], axis = 1)
print(concatenate_item_embed_nd.shape)
concatenate_item_embed = csr_matrix(concatenate_item_embed_nd)

(728, 29300)


In [37]:

# Compute the cosine similarity matrix
print(concatenate_item_embed_nd.shape)
cosine_sim2 = linear_kernel(concatenate_item_embed, concatenate_item_embed)
#Output the shape of tfidf_matrix
cosine_sim2.shape

(728, 29300)


(728, 728)

In [348]:
import pickle
tfidf_save = {"TfidfVectorizer": tfidf, "Tfidf_matrix": tfidf_matrix, "lightFM_embeding_matrix": item_embed_FM, "concatenate_matrix": concatenate_item_embed}
with open('content_embed/all_jieba.pickle', 'wb') as f:
    pickle.dump(tfidf_save, f)

TypeError: cannot pickle '_thread.RLock' object

# According to course similarity to Predict Subgroups

In [38]:
subgroups_df = pd.read_csv('./data/subgroups.csv')
subgroup_num = len(subgroups_df)

subgroups2idx = {}
for (id, name) in zip(subgroups_df["subgroup_id"], subgroups_df["subgroup_name"]):
    subgroups2idx.update({name: id})
idx2subgroups = {v : k for k, v in subgroups2idx.items()}

In [39]:
course2subgroups = {}
i=0
for (course_id, sub_groups) in zip(course_df["course_id"], course_df["sub_groups"]):
    i=i+1
    #print(i)
    try:
        if pd.isnull(sub_groups):
            course2subgroups.update({course_id: []})
        else:
            course2subgroups.update({course_id: [ subgroups2idx[sub_group] for sub_group in sub_groups.split(',')]})
    except:
        print(i)
        continue
list(course2subgroups.items())[:5]

92
132
198
243
340
517
577
692
713


[('61888e868f154b000781b45a', [1, 2]),
 ('54d5a117065a7e0e00725ac0', [3, 4]),
 ('54d5d9952246e60a009ec571', [3, 5]),
 ('54d7148a2246e60a009ec588', [3, 6, 5]),
 ('5513e92b38239d10005778e1', [7, 8])]

In [40]:
len(list(course2subgroups.items()))
#5772b51ec2917a110075fcf9

719

In [41]:
def get_recommendations_new(haved_courses_list, cosine_sim, top = 10):
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_subgroups = []
    # Get the scores of the top most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_subgroups) < top:
            if sim_scores[i][0] not in haved_courses_index_list:
                course_id = course_df['course_id'][sim_scores[i][0]]
                try:
                    for subgroup in course2subgroups[course_id]:
                        if subgroup not in recommend_subgroups:
                            recommend_subgroups.append(subgroup)
                except:
                    continue

    return recommend_subgroups

test = ['605ca63e3cd943ce48dc5daa']
print(course_df['course_name'].iloc[[course2id_mapping[course] for course in test]].tolist())  
test_predict = get_recommendations_new(test, cosine_sim2, 50)
[ idx2subgroups[subgroup_id] for subgroup_id in test_predict ]

['NFT 加密藝術｜買幣、發行、交易一次上手！']


['程式理財',
 '量化交易',
 '金融商品',
 '文書處理',
 '資料科學',
 '更多藝術',
 '平面設計',
 '電腦繪圖',
 '投資觀念',
 '理財',
 '程式入門',
 '量化分析',
 '個人品牌經營',
 '創業',
 '心靈成長與教育',
 '數位行銷',
 '更多生活品味',
 '更多職場技能',
 '求職',
 '社群行銷',
 '角色設計',
 '程式語言',
 '應用設計',
 '網頁前端',
 '動態設計',
 '繪畫與插畫',
 '設計理論',
 '文案',
 '更多人文',
 '更多設計',
 '區塊鏈',
 '色彩學',
 '素描',
 '壓力舒緩',
 '後製剪輯',
 '職場溝通',
 '表演藝術',
 '居家',
 '影像創作',
 '攝影理論',
 '更多程式',
 '程式思維',
 '手機程式開發',
 '網頁設計',
 '樂器',
 '介面設計',
 '軟體程式開發與維護',
 '文學',
 '資料彙整',
 '網站架設']

In [42]:
seen_predict_df = pd.read_csv('./data/test_seen_group.csv')
predict_user = seen_predict_df["user_id"].to_list()
print(len(predict_user))


seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
print(len(seen_user_haved_purchased_course))

7205
59737


# Predict and Save

In [43]:
import csv

predict_users = seen_predict_df["user_id"].to_list()
with open("subgroup_predict50jieba_withoutduplicates.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "subgroup"])
    for user_id in predict_users:
        recommend_subgroups_str = [ str(x) for x in get_recommendations_new(seen_user_haved_purchased_course[user_id], cosine_sim2, top = 49)]
        recommend = " ".join(recommend_subgroups_str)
        writer.writerow([user_id, recommend])

# Map50

In [44]:
val_seen_group_df["subgroup"].isna().sum()

33

In [45]:
train_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    train_user_haved_purchased_course[seen_user_id] = train_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list

val_answer_subgroups = {}
val_seen_group_df_fillna = val_seen_group_df.fillna("")
for seen_user_id, subgroup_ids in zip(val_seen_group_df_fillna["user_id"], val_seen_group_df_fillna["subgroup"]):
    if len(subgroup_ids) > 0:
        course_id_list = [ int(x) for x in subgroup_ids.split(' ')]
        val_answer_subgroups[seen_user_id] = val_answer_subgroups.setdefault(seen_user_id, []) + course_id_list
list(val_answer_subgroups.items())[:5]

[('56dae2b74e3ef90900b7bd0e', [37, 71]),
 ('60e66f29be3e3b0006c4db75', [1, 14, 15]),
 ('5c919efb728ddf00208b9b2b', [1, 3, 5, 25, 29, 50, 51, 52, 66, 67, 72, 73]),
 ('5ac115507997a2001e7c3617', [33, 34, 38, 39]),
 ('5f53b84440c5be3bb873a9d3', [30])]

In [46]:
import ml_metrics

answers = []
predictions = []
map50s = []
for user_id in val_seen_group_df["user_id"]:
    prediction = get_recommendations_new(train_user_haved_purchased_course[user_id], cosine_sim2, top = 50)

    prediction_idxs = prediction
    if user_id in val_answer_subgroups.keys():
        answer_idxs = val_answer_subgroups[user_id]
    else:
        answer_idxs = []

    predictions.append(prediction_idxs)
    answers.append(answer_idxs)
    map50s.append(ml_metrics.mapk(predicted= [prediction_idxs], actual= [answer_idxs], k = 50))
    
print(np.mean(map50s))      
map50 = ml_metrics.mapk(predicted= predictions, actual= answers, k = 50)
map50

0.2756189490668947


0.2756189490668947

In [498]:
#None: 0.264229
#English: 0.26766006438937334
#50&100 English:0.27306260085902573
#jieba:0.2731468547608558
#jieba&english:0.2731718694197286

In [ ]:
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['description']+' ' + x['groups+subgroups'] +' '+ x['required_tools']
0.23474066171745542
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['description']+' ' + x['groups+subgroups']
0.2340450303688849
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['groups+subgroups'] +' '+ x['required_tools']
0.2653982701045437
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['groups+subgroups']
0.26983667430602465
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group
0.2687028204540958
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + x['groups+subgroups']
0.27104218224794335
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.27303138276785993
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + x['groups+subgroups']
0.2703762157322092
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.2720080461522149
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.2721515223384041
course_name + ' ' + teacher_intro + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.2684706101227077
course_name + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.24369995589518365
teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.2729634889638815
course_name + ' ' + teacher_intro + ' ' + group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.27406571210875297
course_name + ' ' + teacher_intro + ' ' + teacher_intro + ' ' + group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.27001374330773553
course_name + ' ' + teacher_intro + ' ' +  group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']+ ' ' + x['groups+subgroups']
0.2730791567913772
course_name + ' ' + teacher_intro + ' ' +  group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.27124937165116886
course_name + ' ' + teacher_intro + ' ' +  group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + will_learn + ' ' + x['groups+subgroups']
0.2718254137856198
course_name + ' ' + teacher_intro + ' ' +  group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.26869781765027473
course_name + ' ' + teacher_intro + ' ' +  group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + x['target_group']
0.27052662220402734
course_name + ' ' + course_name + ' ' + teacher_intro + ' ' +  group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.27145472993282727
course_name + ' ' + teacher_intro + ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups']
0.27250718612256275



In [ ]:
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['description']+' ' + x['groups+subgroups'] +' '+ x['required_tools']
0.23103891276258284
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['description']+' ' + x['groups+subgroups']
0.23139752060813493
course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['groups+subgroups']
0.267745546924934
course_name + ' ' + teacher_intro + ' ' + group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.27432673542193203
course_name + ' ' + teacher_intro + ' ' + group + ' ' +group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro + ' ' + teacher_intro
0.26990711100815196
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.2738288792488569
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.27094173264651994
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.2740595487029608
course_name + ' ' + course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.27341883606357265
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.2753971715744306
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.27468312598184047
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.275463509594546
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.2751253513426604
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.27547246674335063
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.275472247215274
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro + ' ' +target_group
0.2744330886747651

In [ ]:
沒有重複項
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.27555667903950953
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro + ' ' + x['description']
0.23662565145730194
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] +' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.2756189490668947

In [ ]:
沒有重複項
新增去標點符號
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.23573587562593873
去標點符號與英文
course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] + ' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
0.22282753739771333

In [209]:
0.2703762157322092-0.24369995589518365

0.026676259837025568